# 0, 1 задание

## Декапитализация

Можно еще добавить имена, есть функция, которая находится наиболее частые:

In [52]:
import re
from collections import Counter

def extract_potential_names(text):
    # Предложение заканчивается на . ! ? или перевод строки, затем пробелы
    # Ищем слова с заглавной буквы не в начале предложения
    matches = re.findall(r'(?<![\.!\?\n]\s)(?<!^)\b[A-Z][a-z]+\b', text)
    return matches

def most_common_names(text, top_n=30):
    names = extract_potential_names(text)
    counter = Counter(names)
    return counter.most_common(top_n)

with open('Crime and Punishment ascii.txt', 'r', encoding='ascii') as f:
    original_text = f.read()

common_names = most_common_names(original_text, top_n=10)

print("Наиболее часто встречающиеся имена:")
for name, count in common_names:
    print(f"{name}: {count}")

Наиболее часто встречающиеся имена:
Raskolnikov: 408
Petrovitch: 272
Ivanovna: 268
Sonia: 256
Dounia: 234
Razumihin: 212
Katerina: 165
Porfiry: 138
Svidrigalov: 124
Pyotr: 123


В моей реализации это не добавлено

In [ ]:
import re
import json
from collections import defaultdict

# Преобразование в битовый массив исключений 
def build_exception_bit_array(text: str) -> list[int]:
    bit_array = [0] * len(text)
    text_lower = text.lower()

    sentence_end = {'.', '!', '?', '\n'}

    i = 0
    while i < len(text):
        actual_char = text[i]
        expected_char = text_lower[i]  # по умолчанию — маленькая буква

        # --- Правило 1: первая буква текста ---
        if i == 0 and actual_char.isalpha():
            expected_char = actual_char.upper()

        # --- Правило 2: буква после .!? + любые пробелы/переводы строки ---
        if actual_char.isalpha():
            j = i - 1
            while j >= 0 and text[j] in {' ', '\n'}:
                j -= 1
            if j >= 0 and text[j] in sentence_end:
                expected_char = actual_char.upper()

        # --- Правило 3: после двух заглавных ---
        if i >= 2 and text[i - 2].isupper() and text[i - 1].isupper() and actual_char.isalpha():
            expected_char = actual_char.upper()

        # --- Правило 4: одинокая I между не-буквами ---
        if actual_char == 'i':
            left = i - 1
            right = i + 1
            while left >= 0 and not text[left].isalpha():
                left -= 1
            while right < len(text) and not text[right].isalpha():
                right += 1
            if (left < 0 or not text[left].isalpha()) and (right >= len(text) or not text[right].isalpha()):
                expected_char = 'I'

        # --- Сравнение: если не совпадает — исключение ---
        if actual_char != expected_char:
            bit_array[i] = 1

        i += 1

    return bit_array

# Обратное преобразование 
def restore_text_from_exceptions(text_lower: str, bit_array: list[int]) -> str:
    actual_text = []
    sentence_end = {'.', '!', '?', '\n'}

    i = 0
    while i < len(text_lower):
        actual_char = text_lower[i]

        # --- Правило 1: первая буква текста ---
        if i == 0:
            actual_char = actual_char.upper()

        # --- Правило 2: буква после .!? + любые пробелы/переводы строки ---
        if actual_char.isalpha():
            j = i - 1
            while j >= 0 and text_lower[j] in {' ', '\n'}:
                j -= 1
            if j >= 0 and text_lower[j] in sentence_end:
                actual_char = actual_char.upper()

        # --- Правило 3: после двух заглавных ---
        if i >= 2 and actual_text[i - 2].isupper() and actual_text[i - 1].isupper() and actual_char.isalpha():
            actual_char = actual_char.upper()

        # --- Правило 4: одинокая I между не-буквами ---
        if actual_char == 'i':
            left = i - 1
            right = i + 1
            while left >= 0 and not text_lower[left].isalpha():
                left -= 1
            while right < len(text_lower) and not text_lower[right].isalpha():
                right += 1
            if (left < 0 or not text_lower[left].isalpha()) and (right >= len(text_lower) or not text_lower[right].isalpha()):
                actual_char = actual_char.upper()

        # --- Меняем если это исключение ---
        if bit_array[i] == 1:  
            actual_char = actual_char.lower() if actual_char.isupper() else actual_char.upper()
        
        actual_text.append(actual_char)

        i += 1

    return ''.join(actual_text)


# Сохранение в json
def save_results(bit_array, rules_meta, filename):
    output = {
        'bit_array': bit_array,
        'rules': rules_meta,
        'statistics': {
            'total_chars': len(bit_array),
            'exceptions': sum(bit_array),
            'compression_ratio': sum(bit_array) / len(bit_array) if len(bit_array) > 0 else 0
        }
    }

    with open(filename, 'w') as f:
        json.dump(output, f, indent=2)

In [29]:
file_path = "C:\\Users\\User\\Studies\\Кодирование\\CMDC\\task2\\Crime and Punishment ascii.txt"
# file_path = "C:\\Users\\User\\Studies\\Кодирование\\CMDC\\task2\\test1.txt"


with open(file_path, 'r') as f:
    content = f.read()

# Генерируем битовый массив
bit_array = build_exception_bit_array(content)

reconstructed = restore_text_from_exceptions(content.lower(), bit_array)

# Метаданные о правилах
rules_meta = {
    "standard_rules": [
        "First character of text",
        "After sentence-ending punctuation (.!?) followed by whitespace",
        "Third consecutive uppercase letter",
        "Lone 'I' between non-letters"
    ],
    "custom_rules": []
}

# Сохраняем все в json
save_results(bit_array, rules_meta, "capitalization_rules.json")

# Проверка
print(content == reconstructed)

# print(json.dumps(content))
# print(json.dumps(reconstructed))
# print(bit_array)

True


## Арифмитическое кодирование

### Первый способ

In [47]:
!pip install arithmetic-compressor

In [62]:
from arithmetic_compressor import AECompressor
from arithmetic_compressor.models import StaticModel

model = StaticModel({'0': 1125768, '1': 10717})
coder = AECompressor(model)

data = "".join(map(str, bit_array))

compressed = coder.compress(data)
decoded = coder.decompress(compressed, len(data))

print("Проверка декодера :", list(map(int, decoded)) == bit_array)
print("Доля полученного от исходного :", len(compressed) / len(data))

Проверка декодера : True
Доля полученного от исходного : 0.07698913756010858


### Второй способ

In [50]:
import math

# перерассчёт интервала с учётом новой частоты символов
def update_intervals(inter, count, alph, info_let, max_num):
  numAlph = len(alph)
  sumP = 0
  for i in range(numAlph-1):
    id = info_let[alph[i]][0]
    P = info_let[alph[i]][1] / count
    sumP += P
    high = math.ceil(sumP * max_num)

    inter[id][1] = high - 1
    inter[id+1][0] = high

# подсчёт повторяющихся первых бит
def find_common_leading_bits(num1, num2, N):
    common_bits = []
    for i in range(N):
        mask = (1 << (N - i - 1))
        bit1 = num1 & mask
        bit2 = num2 & mask
        if bit1 == bit2:
          if (bit1 > 0):
            common_bits.append("1")
          else:
            common_bits.append("0")
        else:
            return ''.join(common_bits), len(common_bits)

    return ''.join(common_bits), len(common_bits)

# подсчёт разницы границ интервалов
def check_interval_scaling(num1, num2, N):
    count = 0
    mask = (1 << (N - 1))
    if ((num1 & mask) != (num2 & mask)):
      for i in range(N-1):
        mask = (1 << (N - i - 2))
        bit1 = num1 & mask
        bit2 = num2 & mask
        if bit1 != 0 and bit2 == 0:
          count += 1
        else:
          return count

    return count

# создать список уникальных символов и словарь индексов букв
def create_alph_and_info(T):
    # Создаем список уникальных символов в порядке их первого появления
    alph = []
    for char in T:
        if char not in alph:
            alph.append(char)

    # Создаем словарь place
    info_let = {char: [index, 0] for index, char in enumerate(alph)}

    return alph, info_let

def print_interval_binary(low, high, bit_precision, label=""):
    """Выводит интервал в бинарном виде."""
    bin_low = bin(low)[2:].zfill(bit_precision)
    bin_high = bin(high)[2:].zfill(bit_precision)
    print(f"{label}: Low = {low}({bin_low}), High = {high}({bin_high})")


def arithmetic_encode(
    text,
    bit_precision,
    alphabet=None, 
    symbol_info=None,  
    static_model=False, 
    verbose=False
):  
    # изучение алфавита, если не передан
    if (alphabet == None):
        alphabet, symbol_info = create_alph_and_info(text)

    max_num = 2 ** bit_precision
    intervals = [[0, 0] for _ in range(len(alphabet))]  # Пример интервалов
    intervals[-1] = [0, max_num-1]
    
    # Маски для битовых операций
    full_mask = (1 << bit_precision) - 1
    lower_mask = (1 << (bit_precision - 1)) - 1
    highest_bit = 1 << (bit_precision - 1)

    # Инициализация частот символов
    total_symbols = 0
    if static_model:
        # Сброс и пересчет частот для статической модели
        for symbol in text:
            symbol_info[symbol][1] = 0
        for symbol in text:
            symbol_info[symbol][1] += 1
            total_symbols += 1
        update_intervals(intervals, total_symbols, alphabet, symbol_info, max_num)
    else:
        # Инициализация частот для адаптивной модели
        for symbol in alphabet:
            symbol_info[symbol][1] = 1
            total_symbols += 1

    # Начальные границы интервала
    low = 0
    high = max_num - 1
    pending_bits = 0
    encoded_bits = ""

    for symbol in text:
        if verbose:
            print(f"Символ: {symbol}")

        # Обновление интервалов для адаптивной модели
        if not static_model:
            update_intervals(intervals, total_symbols, alphabet, symbol_info, max_num)
            if verbose:
                print(f"Обновлённые интервалы: {intervals}")
            total_symbols += 1
            symbol_info[symbol][1] += 1

        # Вычисление нового интервала для символа
        symbol_id = symbol_info[symbol][0]
        interval_start, interval_end = intervals[symbol_id]
        range_size = high - low + 1

        new_low = low + math.ceil((range_size * interval_start) / max_num)
        new_high = low + math.ceil((range_size * (interval_end + 1)) / max_num) - 1

        if verbose:
            print_interval_binary(new_low, new_high, bit_precision, "Новый интервал")

        # Поиск общих старших битов
        common_bits, shift_amount = find_common_leading_bits(new_low, new_high, bit_precision)
        encoded_bits += common_bits

        if shift_amount > 0:
            # Добавление отложенных битов и сдвиг интервала
            encoded_bits += "1" * pending_bits
            pending_bits = 0

            new_low = (new_low << shift_amount) & full_mask
            new_high = ((new_high << shift_amount) | ((1 << shift_amount) - 1)) & full_mask

            if verbose:
                print_interval_binary(new_low, new_high, bit_precision, "Интервал после совпадающих битов")

        # Проверка на узкий интервал (E1, E2, E3)
        scale_bits = check_interval_scaling(new_low, new_high, bit_precision)
        if scale_bits > 0:
            # Масштабирование интервала
            saved_bit_low = new_low & highest_bit
            new_low = ((new_low << scale_bits) & lower_mask) | saved_bit_low

            saved_bit_high = new_high & highest_bit
            new_high = ((new_high << scale_bits) & lower_mask) | saved_bit_high | ((1 << scale_bits) - 1)

            if verbose:
                print_interval_binary(new_low, new_high, bit_precision, "Интервал после расширения")

        pending_bits += scale_bits

        if verbose:
            print(f"Закодированный текст: {encoded_bits}, Bits: {pending_bits}\n")

        low, high = new_low, new_high

    # if pending_bits > 0:
    #     encoded_bits += "1" * pending_bits

    return encoded_bits


text = "".join(map(str, bit_array))
result = arithmetic_encode(text, 8, alphabet=['0', '1'], symbol_info={'0': [0, 1125768], '1': [1, 10717]}, static_model=True, verbose=False)
print(len(text))
print("Доля полученного от исходного :", len(result) / len(text))

1136485
Доля полученного от исходного : 0.07987522932550803


# 2 задание